In [ ]:
# Install YOLOv5
%cd yolov5
!pip install -r requirements.txt
! pip install torch-pruning
# Import Libraries
import torch
from yolov5 import detect, val
from IPython.display import Image, display

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


## 1. Retraining yolov5 on Pascal VOC

In [ ]:
# #retraining a pretrained yolo model for pascal voc dataset
!python train.py --data VOC.yaml --weights /content/best.pt --epochs 2 --cache --img 416 --project VOC --name yolov5s_pascal

2024-06-24 12:53:42.412096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 12:53:42.412165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 12:53:42.414637: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/best.pt, cfg=, data=VOC.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, op

In [ ]:
#retrained model path 
model_path = ''

In [ ]:
#checking the model config
m = torch.load(model_path)
m['model'].yaml

{'nc': 80,
 'depth_multiple': 0.33,
 'width_multiple': 0.5,
 'anchors': [[10, 13, 16, 30, 33, 23],
  [30, 61, 62, 45, 59, 119],
  [116, 90, 156, 198, 373, 326]],
 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 3, 'C3', [128]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 6, 'C3', [256]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 9, 'C3', [512]],
  [-1, 1, 'Conv', [1024, 3, 2]],
  [-1, 3, 'C3', [1024]],
  [-1, 1, 'SPPF', [1024, 5]]],
 'head': [[-1, 1, 'Conv', [512, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [256, 1, 1]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 3, 'C3', [256, False]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 14], 1, 'Concat', [1]],
  [-1, 3, 'C3', [512, False]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 10], 1, 'Concat', [1]],
  [-1, 3, 'C3', [1024, False]],
  [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]

## 2. Calculating Baseline MAP Scores.

In [ ]:
!python val.py --weights /content/yolov5/yolov5/VOC/yolov5s_pascal/weights/best.pt --data VOC.yaml --img 416

val: data=/content/yolov5/yolov5/data/VOC.yaml, weights=['/content/yolov5/yolov5/VOC/yolov5s_pascal/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-331-gab364c98 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
val: Scanning /content/yolov5/datasets/VOC/labels/test2007.cache... 4952 images, 0 backgrounds, 0 corrupt: 100% 4952/4952 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 155/155 [01:09<00:00,  2.23it/s]
                   all       4952      12032      0.794      0.731      0.805      0.513
             aeroplane       4952        285      0.694     

## 3. Pruning the model

In [ ]:
#
!python prune.py --data VOC.yaml --weights /content/best.pt --epochs 2 --cache --img 416 --project VOC --name prune_yolo

In [ ]:
import torch
from pathlib import Path
from models.common import DetectMultiBackend  # Ensure this path is correct
from utils.general import check_img_size, non_max_suppression, scale_boxes
from utils.torch_utils import select_device
from utils.augmentations import letterbox
import cv2
import numpy as np

In [ ]:
# Loding the pruned model
model = DetectMultiBackend('/content/best.pt', data= 'VOC.yaml')

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs


In [ ]:
def get_channel_sizes(model):
    channel_sizes = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            channel_sizes.append(module.out_channels)

    print(channel_sizes)
    return channel_sizes

channel_sizes = get_channel_sizes(model)

[32, 64, 32, 32, 64, 32, 32, 128, 64, 64, 128, 64, 64, 64, 64, 256, 128, 128, 256, 128, 128, 128, 128, 128, 128, 512, 256, 256, 512, 256, 256, 256, 512, 256, 128, 128, 256, 128, 128, 128, 64, 64, 128, 64, 64, 128, 128, 128, 256, 128, 128, 256, 256, 256, 512, 256, 256, 75, 75, 75]


In [ ]:
!python val.py --weights prune_path --data VOC.yaml --img 416